# RESOLVE01 学習データの生成・DNNモデルの構築
松本(2020)が再現できる範囲

In [7]:
%pwd # ~/ch01_generate_datasetsならOK

'/Users/quetzal/GL/geophysics/Repos/DLXEM/environment'

## 1.1 データ生成

- import

In [8]:
import numpy as np
import pandas as pd
import sys, os
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

sys.path.append('../')
from dlxem import emexecutor as em
from dlxem import networks

- settings

In [9]:
config = {
    'thicks' : [1 for i in range(100)],
    'res_range' : (1e-2, 1e4),
    'height_range' : (15, 75),
    'freqs' : [382, 1822, 7970, 35920, 130100],
    'span' : 7.86,
    'to_ppm' : True,
    'add_noise' : True,
    'noise_level' : [10, 10, 20, 40, 50],
    'random_mode' : 'smooth_mix',
    'ground_char' : 'r'
}

size = 20000
parallel_num = 8
file_name = 'DB/dataset/20210625_a_{}_resolve.csv'.format(size)

- run

In [10]:
%% time
if os.path.exists(file_name):
    print('Datasets already exist')
else:
    resolve = em.Resolve1D(**config)
    data = resolve.multi_process(size, parallel_num)

    df = pd.DataFrame(data)
    df.to_csv(file_name, header=None, index=False)
    print("-> /" + file_name)

Datasets already exist


## 1.2 モデル学習

In [12]:
#全体読み込み
data_file_name = file_name
df = pd.read_csv(data_file_name)

#分割
x = df.iloc[:, :11].values
y = df.iloc[:, 11:].values
y = np.log10(y)
x_train, x_val_test, y_train, y_val_test = train_test_split(x, y, test_size=0.02, random_state=0)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=0.5, random_state=0)

#x正規化
sc = preprocessing.StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)
x_val = sc.transform(x_val)

In [13]:
input_dim = x.shape[1]
output_dim = y.shape[1]

model = networks.get_dnn(input_dim, output_dim)
history = model.fit(x_train, y_train, batch_size=64, epochs=100, verbose=1, validation_data=(x_val, y_val))


Train on 19599 samples, validate on 200 samples
Epoch 1/100
19599/19599 [==============================] - 16s 817us/step - loss: 1.9917 - val_loss: 2.0686
Epoch 2/100
19599/19599 [==============================] - 15s 749us/step - loss: 1.7074 - val_loss: 1.8490
Epoch 3/100
19599/19599 [==============================] - 15s 751us/step - loss: 1.5847 - val_loss: 1.7651
Epoch 4/100
19599/19599 [==============================] - 15s 751us/step - loss: 1.4813 - val_loss: 2.2942
Epoch 5/100
19599/19599 [==============================] - 15s 752us/step - loss: 1.4222 - val_loss: 1.6538
Epoch 6/100
19599/19599 [==============================] - 15s 749us/step - loss: 1.3938 - val_loss: 1.5878
Epoch 7/100
19599/19599 [==============================] - 15s 754us/step - loss: 1.3557 - val_loss: 1.5920
Epoch 8/100
19599/19599 [==============================] - 15s 741us/step - loss: 1.3459 - val_loss: 1.9660
Epoch 9/100
19599/19599 [==============================] - 14s 737us/step - loss: 1.322

OSError: Unable to create file (unable to open file: name = '../data/dnn_history/20210625_model.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 602)

In [15]:
dnnsave_path = 'DB/dnn_history/'
history_name = '20210625a_history.csv'
model_name = '20210625a_model.h5'

model.save(dnnsave_path + model_name)
hist_df = pd.DataFrame(history.history)
hist_df.to_csv(dnnsave_path + history_name)